## Предсказание пола/возраста по посещениям сайтов

Для начала, подключим хранилище s3, содержащее исходные данные:

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import numpy as np
from sklearn.multioutput import MultiOutputClassifier
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch


In [3]:
!chmod 600 .s3_passwd
!mkdir -p mnt
!s3fs hsevkhack mnt -o url=http://hb.vkcs.cloud -o use_path_request_style -o passwd_file=.s3_passwd -o ro

"chmod" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.
�������� ��� 䠩� -p 㦥 �������.
�訡�� �� �६� ��ࠡ�⪨: -p.
�������� ��� 䠩� mnt 㦥 �������.
�訡�� �� �६� ��ࠡ�⪨: mnt.
"s3fs" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


Убедимся, что в директории `mnt` содержатся соответствующие таблицы:

In [4]:
!ls mnt/

"ls" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


Если по каким-то причинам директорию смонтировать не удалось, используйте код ниже, чтобы скачать данные к себе в локальное хранилище. В противном случае **пропустите следующую ячейку**.

In [5]:
import subprocess
!mkdir -p data
!wget https://hsehack.hb.ru-msk.vkcs.cloud/geo_dataframe.csv -P data
!wget https://hsehack.hb.ru-msk.vkcs.cloud/train_users.csv -P data
!mkdir -p data/requests
for i in range(30):
    print(f"Downloading part {i}...")
    subprocess.call(["wget", f"https://hsehack.hb.ru-msk.vkcs.cloud/requests/part_{i}.parquet"])

�������� ��� 䠩� -p 㦥 �������.
�訡�� �� �६� ��ࠡ�⪨: -p.
�������� ��� 䠩� data 㦥 �������.
�訡�� �� �६� ��ࠡ�⪨: data.


"wget" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.
"wget" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.
�訡�� � ᨭ⠪�� �������.


FileNotFoundError: [WinError 2] Не удается найти указанный файл

## Загружаем данные

Используем Pandas для работы с таблицами. `geo_dataframe` содержит данные о геолокации - регионе и стране.

In [6]:
geo = pd.read_csv('mnt/geo_dataframe.csv')
geo

,geo_id,region_id,country_id
0,1,157,40
1,2,161,40
2,3,265,54
3,4,122,54
4,5,78,40
...,...,...,...
5528,5529,246,54
5529,5530,-1,63
5530,5531,101,40
5531,5532,21,68


Таблица `train_users` - это ваша обучающая выборка, содержащая пол и возраст пользователей. Аналогичная таблица для тестового датасета будет вам предоставлена за час до stop coding.

In [7]:
users = pd.read_csv('mnt/train_users.csv')
users

,user_id,gender,age
0,2,1,61
1,3,1,55
2,6,0,46
3,14,0,66
4,17,0,53
...,...,...,...
4999995,17588859,1,64
4999996,17588860,0,69
4999997,17588861,1,51
4999998,17588864,0,30


Для примера - вот распределение пользователей по полу:

In [ ]:
users.groupby('gender').agg({ 'user_id' : len }).plot.bar()

Основные данные о посещениях пользователями сайтов содержатся в табличке `requests`, представленной в формате parquet. Вы можете загрузить только одну часть таблички, или же всю таблицу, если она поместиться в памяти:

Black data


In [ ]:
import os
frames = pd.DataFrame()
users_dict = pd.DataFrame()
users_requests = dict()

for name in os.listdir(r'C:\Users\32233\Downloads\vkhack-main\vkhack-main\gender_prediction\mnt\requests'):
    req = pd.read_parquet(r'C:\\Users\\32233\\Downloads\\vkhack-main\\vkhack-main\\gender_prediction\\mnt\\requests\\'+name)
    frames = pd.concat([frames,req])
    # users request analys

    for i in req['user_id']:
        users_requests[i] = users_requests.get(i, 0) + 1

users_dict = pd.DataFrame(users_requests.items())
display(users_dict)
display(frames)


In [ ]:
users_dict.to_csv('users_requests')

In [ ]:
frames.to_csv('all_data')

## Дерзайте!

Вам необходимо построить предсказательную модель для прогнозирования пола и возраста пользователей по их посещениям. Таблица с тестовыми пользователями будет предоставлена вам за час до stop coding.

ВАЖНО:
* Таблицу с тестовыми данными нельзя использовать для обучения модели. Если жюри увидит, что вы использовали табличку с тестовыми данными - точность модели будет оцениваться в 0 баллов.
* В ходе выступления необходимо продемонстрировать точность модели на тестовых данных.

In [ ]:
# users request analys
users_requests = dict()
for i in frames['user_id']:
    users_requests[i] = users_requests.get(i,0) + 1
users_dict = pd.DataFrame(users_requests.items())
users_dict

In [ ]:

plt.plot([i for i in range(len(users_requests))],users_requests.values())

In [ ]:
users

In [ ]:
# users analys
ages = dict()
gender = dict()
for i in users['age']:
    ages[i] = ages.get(i,0) + 1
for i in users['gender']:
    gender[i] = gender.get(i,0) + 1
df_ages =  pd.DataFrame(ages.items())
df_genders = pd.DataFrame(gender.items())
df_ages = df_ages.sort_index()
display(df_ages)
plt.bar(df_ages[0],df_ages[1])
display(df_genders)
plt.plot(df_genders)

In [ ]:
plt.plot(df_ages[1])

In [ ]:
plt.bar(df_ages[0],df_ages[1])

In [8]:
users_up10 = users.where(users['age']>10)
users_up10

,user_id,gender,age
0,2,1,61
1,3,1,55
2,6,0,46
3,14,0,66
4,17,0,53
...,...,...,...
4999995,17588859,1,64
4999996,17588860,0,69
4999997,17588861,1,51
4999998,17588864,0,30


In [ ]:
frames_up10 = frames[frames['user_id'].isin(users_up10['user_id'])]
# print(frames[(not frames['user_id'].isin(users_up10['user_id']))])
frames_up10

In [ ]:
#fake check
frames_up10_geo = frames_up10[frames_up10['geo_id']>0]
print(frames_up10[frames_up10['geo_id']==-1])
frames_up10_geo

In [ ]:
# users analys
ages = dict()
gender = dict()
for i in users_up10['age']:
    ages[i] = ages.get(i,0) + 1
for i in users['gender']:
    gender[i] = gender.get(i,0) + 1
df_ages =  pd.DataFrame(ages.items())
df_genders = pd.DataFrame(gender.items())
df_ages = df_ages.sort_index()
display(df_ages)
plt.bar(df_ages[0],df_ages[1])


In [ ]:
display(df_genders)
plt.plot(df_genders[1])

In [ ]:
import re
# req = pd.read_parquet(f'shared/data/requests/part_0.parquet')
devices = []
for x in frames_up10_geo['user_agent']:     
        
    pattern = r'\(([^)]+)\)'

    # Поиск первой пары круглых скобок
    match = re.search(pattern, x)

    # Извлечение содержимого из первой пары круглых скобок
    if match:
        result = match.group(1)
        devices.append(result.split()[0])

devices_ = dict()

for el in list(set(devices)):
    if devices.count(el) >= 5000:
        devices_[el] = devices.count(el)
    
plt.bar(range(len(devices_)), list(devices_.values()), tick_label=list(devices_.keys()))
plt.xticks(rotation=90)

plt.show()

devices_


In [ ]:
users_requests = dict()
for i in frames_up10['user_id']:
    users_requests[i] = users_requests.get(i,0) + 1
users_dict = pd.DataFrame(users_requests.items())
users_dict

In [ ]:
import matplotlib.pyplot as plt
plt.plot([i for i in range(len(users_requests))],users_requests.values())

In [ ]:
frames_up10.to_csv('frames_up10')

In [ ]:
# Read in the two CSV files
df2 = pd.read_csv(r"C:\Users\32233\Downloads\vkhack-main\vkhack-main\gender_prediction\mnt\geo_dataframe.csv")

# Merge the two CSV files based on user_id
merged_df = pd.merge(frames_up10, df2, on='geo_id')
users_df = pd.read_csv(r"C:\Users\32233\Downloads\vkhack-main\vkhack-main\gender_prediction\mnt\train_users.csv")
all_df = pd.merge(merged_df, users_df, on='user_id')
all_df

In [ ]:
all_df = all_df[all_df['region_id']>0]
all_df

In [ ]:
all_df.to_csv('all_correct_df')

In [ ]:
all_df = all_df[all_df['referer'].str.split('/').len()>3]
all_df

In [ ]:
all_df.to_csv('all_correct_will_sep')

In [ ]:
all_df = pd.read_csv(r"C:\Users\32233\Downloads\vkhack-main\vkhack-main\gender_prediction\all_correct_will_sep",low_memory=False)

In [ ]:
def func(s):
    #print(s)
    q = s.split('/')
    try:
        path = re.search(r'\d+', q[3])
        domain = re.search(r'\d+', q[2])
    except:
        return [0,0]
    
    return [int(domain.group()), int(path.group())]
func('//dso1/123_fds')

In [ ]:
all_df[['host_num', 'path_num']] = req['referer'].apply(func)
all_df


In [ ]:
arr = ['Linux', 'iPhone', 'Windows', 'Macintosh', 'X11']

In [10]:
def func(s):
    domain = 0
    path = 0
    try:
        s = s.split('/')
        path = int(s[-1].split('_')[-1])
        domain = int(s[-2].split('_')[-1])
    except Exception as e:
        #print(s)
        #print(f"Error processing referer: {s}. Error: {e}")
        pass
    return [domain, path]
func("https://www.domain_2945/path_137377")

[2945, 137377]

In [11]:
frames = pd.DataFrame()
k=0
for name in os.listdir(r'C:\Users\32233\Downloads\vkhack-main\vkhack-main\gender_prediction\mnt\requests'):
    req = pd.read_parquet(r'C:\\Users\\32233\\Downloads\\vkhack-main\\vkhack-main\\gender_prediction\\mnt\\requests\\'+name)
    req = pd.merge(req, geo, on='geo_id',how = 'inner')
    req = pd.merge(req, users, on='user_id',how = 'inner')
    
    req2 = req[req['user_id'].isin(users_up10['user_id'])]
    req3 = req2[req2['region_id']>=0]
    req3[['domen','path']]= pd.DataFrame([func(i) for i in req3['referer'].values]).astype(int)
    req3
    frames = pd.concat([frames,req3])
    k+=1
    print(k)
frames

C:\Users\32233\AppData\Local\Temp\ipykernel_18868\684847912.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req3[['domen','path']]= pd.DataFrame([func(i) for i in req3['referer'].values]).astype(int)
C:\Users\32233\AppData\Local\Temp\ipykernel_18868\684847912.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req3[['domen','path']]= pd.DataFrame([func(i) for i in req3['referer'].values]).astype(int)


1


C:\Users\32233\AppData\Local\Temp\ipykernel_18868\684847912.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req3[['domen','path']]= pd.DataFrame([func(i) for i in req3['referer'].values]).astype(int)
C:\Users\32233\AppData\Local\Temp\ipykernel_18868\684847912.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req3[['domen','path']]= pd.DataFrame([func(i) for i in req3['referer'].values]).astype(int)


2


C:\Users\32233\AppData\Local\Temp\ipykernel_18868\684847912.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req3[['domen','path']]= pd.DataFrame([func(i) for i in req3['referer'].values]).astype(int)
C:\Users\32233\AppData\Local\Temp\ipykernel_18868\684847912.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req3[['domen','path']]= pd.DataFrame([func(i) for i in req3['referer'].values]).astype(int)


...

C:\Users\32233\AppData\Local\Temp\ipykernel_18868\684847912.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req3[['domen','path']]= pd.DataFrame([func(i) for i in req3['referer'].values]).astype(int)
C:\Users\32233\AppData\Local\Temp\ipykernel_18868\684847912.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req3[['domen','path']]= pd.DataFrame([func(i) for i in req3['referer'].values]).astype(int)


29


C:\Users\32233\AppData\Local\Temp\ipykernel_18868\684847912.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req3[['domen','path']]= pd.DataFrame([func(i) for i in req3['referer'].values]).astype(int)
C:\Users\32233\AppData\Local\Temp\ipykernel_18868\684847912.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req3[['domen','path']]= pd.DataFrame([func(i) for i in req3['referer'].values]).astype(int)


30


,timestamp,geo_id,referer,user_id,user_agent,region_id,country_id,gender,age,domen,path
0,1712124471,1224,https://domain_611/path_197956,7328826,Mozilla/5.0 (Linux; Android 10; K) AppleWebKit...,30,40,0,75,611.0,197956.0
1,1711941192,1224,https://domain_3216/path_164018,7328826,Mozilla/5.0 (Linux; Android 10; K) AppleWebKit...,30,40,0,75,3216.0,164018.0
2,1711987643,1224,https://www.domain_294/path_177875,6620509,Mozilla/5.0 (Linux; arm_64; Android 13; ZS676K...,30,40,1,68,294.0,177875.0
3,1712150424,1224,https://www.domain_294/path_67805,6620509,Mozilla/5.0 (Linux; arm_64; Android 13; ZS676K...,30,40,1,68,294.0,67805.0
4,1712162696,1224,https://domain_551/path_175610,14089224,Mozilla/5.0 (Linux; arm_64; Android 12; SM-A12...,30,40,0,23,551.0,175610.0
...,...,...,...,...,...,...,...,...,...,...,...
4660790,1712175879,3754,https://domain_1180/path_175610,13303764,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,144,54,1,41,NaN,NaN
4660791,1711934338,4214,https://domain_2194/path_7822,1082865,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5)...,107,68,0,75,NaN,NaN
4660793,1712052746,3655,https://domain_2206/path_175610,14733710,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,86,40,1,52,NaN,NaN
4660794,1712174961,4245,https://domain_609/path_175610,9050544,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,232,68,1,33,NaN,NaN


In [ ]:
frames.to_csv('all_beauty_data.csv')

In [12]:
frames.drop(columns='referer',axis=1)

,timestamp,geo_id,user_id,user_agent,region_id,country_id,gender,age,domen,path
0,1712124471,1224,7328826,Mozilla/5.0 (Linux; Android 10; K) AppleWebKit...,30,40,0,75,611.0,197956.0
1,1711941192,1224,7328826,Mozilla/5.0 (Linux; Android 10; K) AppleWebKit...,30,40,0,75,3216.0,164018.0
2,1711987643,1224,6620509,Mozilla/5.0 (Linux; arm_64; Android 13; ZS676K...,30,40,1,68,294.0,177875.0
3,1712150424,1224,6620509,Mozilla/5.0 (Linux; arm_64; Android 13; ZS676K...,30,40,1,68,294.0,67805.0
4,1712162696,1224,14089224,Mozilla/5.0 (Linux; arm_64; Android 12; SM-A12...,30,40,0,23,551.0,175610.0
...,...,...,...,...,...,...,...,...,...,...
4660790,1712175879,3754,13303764,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,144,54,1,41,NaN,NaN
4660791,1711934338,4214,1082865,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5)...,107,68,0,75,NaN,NaN
4660793,1712052746,3655,14733710,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,86,40,1,52,NaN,NaN
4660794,1712174961,4245,9050544,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,232,68,1,33,NaN,NaN


In [13]:
devices_ = {'Windows': 5568072, 'Linux;': 4174088, 'Macintosh;': 85419, 'X11;': 61863, 'iPhone;': 103331}


def rdt(s):
    pattern = r'\(([^)]+)\)'
    
    match = re.search(pattern, s)
    
    if match:
        return match.group(1).split()[0]
    else:
        return s


devs = list(devices_.keys())
def vd(s):
    ret_val = [0 for _ in range(len(devs))]
    if s in devs:
        ret_val[devs.index(s)] = 1
    return ret_val


frames[devs] = pd.DataFrame([vd(rdt(i)) for i in frames['user_agent'].values]).astype(int)
#frames = frames.drop('user_agent', axis=1)
frames


,timestamp,geo_id,referer,user_id,user_agent,region_id,country_id,gender,age,domen,path,Windows,Linux;,Macintosh;,X11;,iPhone;
0,1712124471,1224,https://domain_611/path_197956,7328826,Mozilla/5.0 (Linux; Android 10; K) AppleWebKit...,30,40,0,75,611.0,197956.0,0,1,0,0,0
1,1711941192,1224,https://domain_3216/path_164018,7328826,Mozilla/5.0 (Linux; Android 10; K) AppleWebKit...,30,40,0,75,3216.0,164018.0,0,1,0,0,0
2,1711987643,1224,https://www.domain_294/path_177875,6620509,Mozilla/5.0 (Linux; arm_64; Android 13; ZS676K...,30,40,1,68,294.0,177875.0,0,1,0,0,0
3,1712150424,1224,https://www.domain_294/path_67805,6620509,Mozilla/5.0 (Linux; arm_64; Android 13; ZS676K...,30,40,1,68,294.0,67805.0,0,1,0,0,0
4,1712162696,1224,https://domain_551/path_175610,14089224,Mozilla/5.0 (Linux; arm_64; Android 12; SM-A12...,30,40,0,23,551.0,175610.0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4660790,1712175879,3754,https://domain_1180/path_175610,13303764,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,144,54,1,41,NaN,NaN,1,0,0,0,0
4660791,1711934338,4214,https://domain_2194/path_7822,1082865,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5)...,107,68,0,75,NaN,NaN,0,1,0,0,0
4660793,1712052746,3655,https://domain_2206/path_175610,14733710,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,86,40,1,52,NaN,NaN,0,1,0,0,0
4660794,1712174961,4245,https://domain_609/path_175610,9050544,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,232,68,1,33,NaN,NaN,0,1,0,0,0


In [ ]:
frames.to_csv('ebat16.csv')

In [14]:
from torch.utils.data import Dataset,DataLoader
from catboost import Pool

#all_df = frames.drop(columns = 'referer',axis=1)
all_df = frames[:10**7]
all_df.drop(columns='timestamp')

target_cols = ['gender']
X = all_df.drop(target_cols, axis=1)
y = all_df[target_cols]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



X_test_array = np.array(X_test)
fearutes = list(X_train.columns)

cat_features = ['referer','user_agent']
#
# # Инициализация и обучение модели CatBoost
model = CatBoostClassifier(iterations=1000, eval_metric='Accuracy',learning_rate=0.5)
model.fit(X_train, y_train, cat_features=cat_features, eval_set=(X_test, y_test))
# multi_output_model = MultiOutputClassifier(model)
# multi_output_model.fit(X_train, y_train, cat_features=cat_features)
# # Предсказание на тестовой выборке
predictions = model.predict(X_test_array)
#
# # Оценка модели
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy}')

0:	learn: 0.6190576	test: 0.6244115	best: 0.6244115 (0)	total: 1.38s	remaining: 22m 55s
1:	learn: 0.6322292	test: 0.6404915	best: 0.6404915 (1)	total: 2.18s	remaining: 18m 7s
2:	learn: 0.6370406	test: 0.6459250	best: 0.6459250 (2)	total: 3.92s	remaining: 21m 43s
3:	learn: 0.6380380	test: 0.6469315	best: 0.6469315 (3)	total: 5.26s	remaining: 21m 50s
4:	learn: 0.6392224	test: 0.6489635	best: 0.6489635 (4)	total: 6.52s	remaining: 21m 38s
5:	learn: 0.6432859	test: 0.6537370	best: 0.6537370 (5)	total: 8.37s	remaining: 23m 6s
6:	learn: 0.6441372	test: 0.6547985	best: 0.6547985 (6)	total: 10.2s	remaining: 24m 12s
7:	learn: 0.6444436	test: 0.6552425	best: 0.6552425 (7)	total: 11.4s	remaining: 23m 32s
8:	learn: 0.6461490	test: 0.6568270	best: 0.6568270 (8)	total: 13.3s	remaining: 24m 19s
9:	learn: 0.6463037	test: 0.6573070	best: 0.6573070 (9)	total: 14.7s	remaining: 24m 14s
10:	learn: 0.6463811	test: 0.6576025	best: 0.6576025 (10)	total: 15.6s	remaining: 23m 19s
11:	learn: 0.6467050	test: 0.657

In [15]:
model.save_model('model69.cbm')

## Убираем за собой

В конце работы можем размонтировать директорию:

In [ ]:
!umount mnt